In [8]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import glob

# Ambil semua file CSV dari dua folder
csv_files_1 = glob.glob("reviews_1/*.csv")
csv_files_2 = glob.glob("reviews_2/*.csv")

# Gabungkan daftar file dari kedua folder
csv_files = csv_files_1 + csv_files_2

# Gabungkan semua CSV jadi satu DataFrame
df_list = [pd.read_csv(file) for file in csv_files]
df = pd.concat(df_list, ignore_index=True)

# Tampilkan jumlah total baris
print(f"Total data: {len(df)} baris")

# Hitung jumlah UserID unik
jumlah_userid = df['UserID'].nunique()
print(f"Jumlah UserID unik: {jumlah_userid}")

Total data: 11339 baris
Jumlah UserID unik: 227


In [2]:
import pandas as pd
import re
import glob
from tqdm import tqdm

# Kamus normalisasi singkat
normalization_dict = {
    "u": "you", "r": "are", "ur": "your", "pls": "please", "plz": "please",
    "thx": "thanks", "ty": "thank you", "btw": "by the way", "lol": "laughing out loud",
    "lmao": "laughing my ass off", "omg": "oh my god", "wtf": "what the fuck",
    "idk": "i do not know", "imo": "in my opinion", "imho": "in my humble opinion",
    "gg": "good game", "ez": "easy", "nerf": "weaken", "buff": "strengthen",
    "noob": "newbie", "grind": "repetitive task", "op": "overpowered",
    "laggy": "slow connection", "gitgud": "get good", "f2p": "free to play",
    "p2w": "pay to win", "dlc": "downloadable content", "npc": "non player character",
    "xp": "experience points", "lvl": "level", "rng": "random number generator",
    "afk": "away from keyboard", "af": "as fuck", "rp": "roleplay",
    "ive": "i have", "tl": "too long", "dr": "did not read"
}

# Deteksi konteks fps
def disambiguate_fps(text):
    if re.search(r"\b(fps)\b", text.lower()):
        if re.search(r"(graphics|frame|lag|performance|per second)", text.lower()):
            text = re.sub(r"\bfps\b", "frames per second", text, flags=re.IGNORECASE)
        elif re.search(r"(shooter|gun|weapon|enemy|combat|first person)", text.lower()):
            text = re.sub(r"\bfps\b", "first person shooter", text, flags=re.IGNORECASE)
    return text

# Fungsi normalisasi + pembersihan ringan
def preprocess_review(text):
    # Pastikan input adalah string, jika tidak, konversi menjadi string kosong
    if not isinstance(text, str):
        text = ""
    
    # Lowercase ringan
    text = text.lower()
    
    # Ganti singkatan yang umum
    words = text.split()
    words = [normalization_dict.get(word, word) for word in words]
    text = " ".join(words)
    
    # Disambiguasi kata "fps"
    text = disambiguate_fps(text)
    
    # Hapus URL dan simbol aneh
    text = re.sub(r"http\S+", "", text)               # hapus url
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)        # hapus karakter aneh
    text = re.sub(r"\s+", " ", text).strip()          # hilangkan spasi berlebih
    
    return text

# Ambil semua file CSV dari dua folder
csv_files_1 = glob.glob("reviews_1/*.csv")
csv_files_2 = glob.glob("reviews_2/*.csv")

# Gabungkan daftar file dari kedua folder
csv_files = csv_files_1 + csv_files_2

# Gabungkan semua CSV jadi satu DataFrame
df_list = [pd.read_csv(file) for file in csv_files]
df = pd.concat(df_list, ignore_index=True)

# Tampilkan jumlah total baris
print(f"Total data: {len(df)} baris")

# Hitung jumlah UserID unik
jumlah_userid = df['UserID'].nunique()
print(f"Jumlah UserID unik: {jumlah_userid}")

# Memproses review
tqdm.pandas()  # Mengaktifkan tqdm untuk progress bar
df['Cleaned_Review'] = df['Review'].progress_apply(preprocess_review)

# Tampilkan hasil pembersihan review (contoh)
print(df[['UserID', 'Review', 'Cleaned_Review']].head())

# Menyimpan DataFrame yang telah diproses ke dalam file CSV baru
df.to_csv("Cleaned_Reviews.csv", index=False)

print("File CSV dengan review yang telah dibersihkan berhasil disimpan sebagai 'Cleaned_Reviews.csv'.")


Total data: 11647 baris
Jumlah UserID unik: 227


100%|██████████| 11647/11647 [00:00<00:00, 21629.92it/s]


              UserID                                             Review  \
0  76561199076918197                                 Better than Naruto   
1  76561199076918197        I am offensive and I find this game Indian.   
2  76561199076918197  1) Game breakin glitches. 2) Out dated physics...   
3  76561199076918197                                Too much of hentai.   
4  76561199076918197  Dude, say whatever you wanna say, but remember...   

                                      Cleaned_Review  
0                                 better than naruto  
1         i am offensive and i find this game indian  
2  1 game breakin glitches 2 out dated physics 3 ...  
3                                 too much of hentai  
4  dude say whatever you wanna say but remember i...  
File CSV dengan review yang telah dibersihkan berhasil disimpan sebagai 'Cleaned_Reviews.csv'.


In [1]:
# Import necessary libraries
import pandas as pd
import re
import glob
from tqdm import tqdm

# Kamus normalisasi singkat
normalization_dict = {
    "u": "you", "r": "are", "ur": "your", "pls": "please", "plz": "please",
    "thx": "thanks", "ty": "thank you", "btw": "by the way", "lol": "laughing out loud",
    "lmao": "laughing my ass off", "omg": "oh my god", "wtf": "what the fuck",
    "idk": "i do not know", "imo": "in my opinion", "imho": "in my humble opinion",
    "gg": "good game", "ez": "easy", "nerf": "weaken", "buff": "strengthen",
    "noob": "newbie", "grind": "repetitive task", "op": "overpowered",
    "laggy": "slow connection", "gitgud": "get good", "f2p": "free to play",
    "p2w": "pay to win", "dlc": "downloadable content", "npc": "non player character",
    "xp": "experience points", "lvl": "level", "rng": "random number generator",
    "afk": "away from keyboard", "af": "as fuck", "rp": "roleplay",
    "ive": "i have", "tl": "too long", "dr": "did not read"
}

# Deteksi konteks fps
def disambiguate_fps(text):
    if re.search(r"\b(fps)\b", text.lower()):
        if re.search(r"(graphics|frame|lag|performance|per second)", text.lower()):
            text = re.sub(r"\bfps\b", "frames per second", text, flags=re.IGNORECASE)
        elif re.search(r"(shooter|gun|weapon|enemy|combat|first person)", text.lower()):
            text = re.sub(r"\bfps\b", "first person shooter", text, flags=re.IGNORECASE)
    return text

# Fungsi normalisasi + pembersihan ringan
def preprocess_review(text):
    if not isinstance(text, str):
        text = ""
    text = text.lower()  # Lowercase ringan
    words = text.split()
    words = [normalization_dict.get(word, word) for word in words]
    text = " ".join(words)
    text = disambiguate_fps(text)
    text = re.sub(r"http\S+", "", text)               # hapus url
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)        # hapus karakter aneh
    text = re.sub(r"\s+", " ", text).strip()          # hilangkan spasi berlebih
    return text

# Ambil semua file CSV dari dua folder
csv_files_1 = glob.glob("reviews_1/*.csv")
csv_files_2 = glob.glob("reviews_2/*.csv")
csv_files = csv_files_1 + csv_files_2

# Gabungkan semua CSV jadi satu DataFrame
df_list = [pd.read_csv(file) for file in csv_files]
df = pd.concat(df_list, ignore_index=True)

# Tampilkan jumlah total baris
print(f"Total data: {len(df)} baris")

# Hitung jumlah UserID unik
jumlah_userid = df['UserID'].nunique()
print(f"Jumlah UserID unik: {jumlah_userid}")

# Memproses review
tqdm.pandas()  # Mengaktifkan tqdm untuk progress bar
df['Cleaned_Review'] = df['Review'].progress_apply(preprocess_review)

# Tampilkan hasil pembersihan review (contoh)
print(df[['UserID', 'Review', 'Cleaned_Review']].head())

# Menyimpan DataFrame yang telah diproses ke dalam file CSV baru
df.to_csv("Cleaned_Reviews_v2.csv", index=False)

print("File CSV dengan review yang telah dibersihkan berhasil disimpan sebagai 'Cleaned_Reviews_v2.csv'.")


Total data: 11647 baris
Jumlah UserID unik: 227


100%|██████████| 11647/11647 [00:00<00:00, 34281.69it/s]


              UserID                                             Review  \
0  76561199076918197                                 Better than Naruto   
1  76561199076918197        I am offensive and I find this game Indian.   
2  76561199076918197  1) Game breakin glitches. 2) Out dated physics...   
3  76561199076918197                                Too much of hentai.   
4  76561199076918197  Dude, say whatever you wanna say, but remember...   

                                      Cleaned_Review  
0                                 better than naruto  
1         i am offensive and i find this game indian  
2  1 game breakin glitches 2 out dated physics 3 ...  
3                                 too much of hentai  
4  dude say whatever you wanna say but remember i...  
File CSV dengan review yang telah dibersihkan berhasil disimpan sebagai 'Cleaned_Reviews_v2.csv'.
